In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from IPython.display import Image, display_png
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

In [2]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)
t = t.T.drop([
    'C00001', # H2O
#     'C00003', # NAD
#     'C00004', # NADH
#     'C00005', # NADPH
#     'C00006', # NADP
    'C00009', # Phosphate
#     'C00010', # CoA
#     'C00011', # CO2
    'C00033', # Acetate
    'C00080', # H+
#     'C00138', # Reduced ferredoxin
#     'C00139', # Oxidized ferredoxin
    'C15602', # Quinone
    'C15603' # Hydroquinone
]).T

In [3]:
h = [
#     'R00200', Pyruvate <- Phosphoenolpyruvate
    'R00268',
#     'R00341', # Oxaloacetate -> Phosphoenolpyruvate
    'R00342',
    'R00351',
    'R00361',
    'R00658',
    'R01015',
    'R01061',
    'R01070',
#     'R01082',
    'R01196',
    'R01197',
    'R01325',
    'R01512',
    'R01518',
    'R01786',
    'R01899',
    'R01900',
#     'R02164',
    'R02740',
    'R04779',
    'R10343'
]

In [4]:
p = t.T[h].T

for i in p:
    if sum(p[i] != 0) == 0:
        p=p.drop(i, axis=1)
        
p['C00007'] = pd.Series(np.zeros(p.shape[0]), index=p.index)
p['C00122'] = pd.Series(np.zeros(p.shape[0]), index=p.index)
p['C00186'] = pd.Series(np.zeros(p.shape[0]), index=p.index)

p = p.T.sort_index().T

for i in list(p.columns):
    url = 'http://rest.kegg.jp/get/{}'.format(i)
    savename = '../html/{}.html'.format(i)
    urllib.request.urlretrieve(url, savename)
    
g = pd.DataFrame()
for i in list(p.columns):
    g = pd.concat([g, pd.read_table('../html/{}.html'.format(i), header=None)], axis=1)
    
c = []
for i in list(g.T[1]):
    c.append(i[12:-1])
    
p['Biomass'] = pd.Series(np.zeros(p.shape[0]), index=p.index)
p = p.T

x = [LpVariable(i) for i in p.index]
c.append('Biomass')
o = pd.DataFrame(x, index=c)

In [5]:
m = LpProblem(sense=LpMaximize)
m += o[0]['Biomass']

capacity_plus = 100
capacity_minus = -100

for i in p.columns:
    m += np.dot(list(p[i]), x) <= capacity_plus
    m += np.dot(list(p[i]), x) >= capacity_minus
    
# NADPH <-> NADH
m += o[0]['NADPH'] + o[0]['NAD+'] - o[0]['NADP+'] - o[0]['NADH'] <= capacity_plus
m += o[0]['NADPH'] + o[0]['NAD+'] - o[0]['NADP+'] - o[0]['NADH'] >= capacity_minus
    
#  Oxophos
m += 5*o[0]['ATP'] + 2*o[0]['NAD+'] + o[0]['CO2'] - 2*o[0]['NADH'] - o[0]['Oxygen'] - 5*o[0]['ADP'] <= capacity_plus
m += 5*o[0]['ATP'] + 2*o[0]['NAD+'] + o[0]['CO2'] - 2*o[0]['NADH'] - o[0]['Oxygen'] - 5*o[0]['ADP'] >= capacity_minus

# Oxaloacetate -> Phosphoenolpyruvate
m += o[0]['CO2'] + o[0]['ADP'] + o[0]['Phosphoenolpyruvate'] - o[0]['ATP'] - o[0]['Oxaloacetate'] <= 0
m += o[0]['CO2'] + o[0]['ADP'] + o[0]['Phosphoenolpyruvate'] - o[0]['ATP'] - o[0]['Oxaloacetate'] >= capacity_minus

# Oxaloacetate <- Phosphoenolpyruvate
m += o[0]['CO2'] + o[0]['Phosphoenolpyruvate']  - o[0]['Oxaloacetate'] <= capacity_plus
m += o[0]['CO2'] + o[0]['Phosphoenolpyruvate']  - o[0]['Oxaloacetate'] >= 0

# Pyruvate <- Phosphoenolpyruvate
m += o[0]['Pyruvate'] + o[0]['ATP'] - o[0]['ADP'] - o[0]['Phosphoenolpyruvate'] <= capacity_plus
m += o[0]['Pyruvate'] + o[0]['ATP'] - o[0]['ADP'] - o[0]['Phosphoenolpyruvate'] >= 0

#  Lactate oxidoreductase
m += o[0]['NADH'] + o[0]['Pyruvate'] - o[0]['NAD+'] - o[0]['(S)-Lactate'] <= 0
m += o[0]['NADH'] + o[0]['Pyruvate'] - o[0]['NAD+'] - o[0]['(S)-Lactate'] >= capacity_minus

#  Fumarate -> (S)-Malate
m += o[0]['(S)-Malate'] - o[0]['Fumarate'] <= capacity_plus
m += o[0]['(S)-Malate'] - o[0]['Fumarate'] >= 0

# # biomass product 
m += (
    o[0]['Biomass'] + 20*o[0]['NADPH'] + 20.650823*o[0]['ADP'] - 2.75194*o[0]['alpha-D-Glucose 6-phosphat'] - 20.704451*o[0]['ATP'] - 7.04451*o[0]['Acetyl-CoA'] - 20*o[0]['NADP+']
)  >= 0

m += (
    o[0]['Biomass'] + 20*o[0]['NADPH'] + 20.650823*o[0]['ADP'] - 2.75194*o[0]['alpha-D-Glucose 6-phosphat'] - 20.704451*o[0]['ATP'] - 7.04451*o[0]['Acetyl-CoA'] - 20*o[0]['NADP+']
)  <= 100

m += o[0]['alpha-D-Glucos'] >= 100
m += o[0]['alpha-D-Glucos'] <= 100
m += o[0]['CO2'] <= 0

m += o[0]['Oxygen'] >= 100
m += o[0]['Oxygen'] <= 100
# m += o[0]['(S)-Lactate'] <= 0
# m += o[0]['(S)-Lactate'] >= capacity_minus

In [6]:
status = m.solve()
print(LpStatus[status])

Unbounded


In [7]:
y = []
for i in x:
    y.append(value(i))

In [8]:
result = pd.DataFrame(y, index=o.index, columns=['value'])
# result.value = result.value/max(result['value'])

In [9]:
result

,value
ATP,4.000000e+01
NAD+,0.000000e+00
NADH,0.000000e+00
NADPH,-1.000000e+02
NADP+,0.000000e+00
Oxygen,1.000000e+02
ADP,0.000000e+00
CoA,1.000000e+02
CO2,0.000000e+00
Pyruvate,0.000000e+00
